In [ ]:
import rasterio
import glob
from matplotlib import pyplot
import numpy as np
import os
import json

periods = glob.glob('output/*_ndwi_clipped.tif')
all_samples = { }

for period in periods:
    period = period.replace('output/', '').replace('_ndwi_clipped.tif', '')
    all_samples[period] = []
    
    with rasterio.open(f'output/{period}_ndwi_clipped.tif', 'r') as src:
        data = src.read(1)
        data = data[::8,::8]
        
    value_range = [-1, 1]
    bin_step = 0.1
    bin_count = 20

    class NpEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            if isinstance(obj, np.floating):
                return float(obj)
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return json.JSONEncoder.default(self, obj)

    v = value_range[0]
    while v < value_range[1]-bin_step:
        v_min = int(v * 127 + 128)
        v_max = int((v + bin_step) * 127 + 128)

        values = np.argwhere((data >= v_min) & (data < v_max))
        if values.shape[0] >= bin_count:
            samples = np.random.choice(np.arange(0,values.shape[0]), size=bin_count)
            for s in samples:
                all_samples[period].append({
                    'pos': list(values[s]),
                    'water': None,
                    'value': data[values[s][0],values[s][1]]
                })

        v += bin_step

with open('samples.json', 'w') as f:
    f.write(json.dumps(all_samples, indent=4, cls=NpEncoder))